<a href="https://colab.research.google.com/github/sibu1234/Deep-Learning/blob/master/CNN_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.2.0


In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [0]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [0]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images.shape

(60000, 28, 28)

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        51264     
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1228848   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,281,434
Trainable params: 1,281,434
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.2, initial_epoch=3)

Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1880 - accuracy: 0.9305 - val_loss: 0.2279 - val_accuracy: 0.9183
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1522 - accuracy: 0.9428 - val_loss: 0.2324 - val_accuracy: 0.9178
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1242 - accuracy: 0.9532 - val_loss: 0.2484 - val_accuracy: 0.9141
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0988 - accuracy: 0.9627 - val_loss: 0.2726 - val_accuracy: 0.9151
Epoch 8/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0781 - accuracy: 0.9712 - val_loss: 0.2904 - val_accuracy: 0.9165
Epoch 9/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0658 - accuracy: 0.9748 - val_loss: 0.3454 - val_accuracy: 0.9133
Epoch 10/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.0524 - accuracy: 0.9803 - val_loss: 0.3722 - val_accuracy